In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt

# Load the crime data
crime_data_path = "data/2024-07-west-yorkshire-street.csv"
crime_df = pd.read_csv(crime_data_path)

crime_df.head()

In [ ]:
!pip freeze > requirements.txt

In [ ]:
import geopandas

leeds = geopandas.read_file("data/osm_leeds.geojson")

leeds

In [3]:
leeds = leeds.set_index("name")

In [ ]:
leeds["area"] = leeds.area
leeds["area"]

In [ ]:
leeds["boundary"] = leeds.boundary
leeds["boundary"]

In [ ]:
leeds["centroid"] = leeds.centroid
leeds["centroid"]

In [ ]:
first_point = leeds["centroid"].iloc[0]
leeds["distance"] = leeds["centroid"].distance(first_point)
leeds["distance"]

In [ ]:
leeds["distance"].mean()

In [ ]:
leeds.plot("area", legend=True)

In [ ]:
leeds.explore("area", legend=False)

In [ ]:
leeds.info()

In [ ]:
# Load the Leeds map from GeoJSON
leeds_map_path = "data/osm_leeds.geojson"
leeds_gdf = geopandas.read_file(leeds_map_path)

# Display the first few rows to understand the structure
leeds_gdf.head()

In [ ]:
unique_highway_fields = leeds_gdf["highway"].unique()
unique_highway_fields

In [ ]:
# Create a folium map centred around Leeds
leeds_map = folium.Map(location=[53.8008, -1.5491], zoom_start=12)

# Extract the relevant columns from the crime data for latitude and longitude
crime_locations = crime_df[["Latitude", "Longitude"]].dropna()

# Create a list of locations for the heatmap
heatmap_data = crime_locations.values.tolist()

# Add HeatMap to the Leeds map using green (safe) to red (dangerous) colours
HeatMap(
    heatmap_data,
    min_opacity=0.4,
    radius=10,
    blur=15,
    max_zoom=1,
    gradient={0.2: "green", 0.5: "yellow", 1: "red"},
).add_to(leeds_map)

# Overlay the Leeds boundaries from the GeoJSON file
folium.GeoJson(leeds_gdf, name="Leeds Boundaries").add_to(leeds_map)

# Save the map as an HTML file to view
leeds_map.save("output/leeds_crime_heatmap.html")

# Display the map object
leeds_map

In [ ]:
import glob
import os

# Define the directory containing the CSV files
data_directory = "data/"

# Use glob to find all CSV files that contain 'west-yorkshire-street' in their name
csv_files = glob.glob(os.path.join(data_directory, "*west-yorkshire-street*.csv"))

# Load and concatenate all CSV files into a single DataFrame
combined_crime_df = pd.concat(
    [pd.read_csv(file) for file in csv_files], ignore_index=True
)

# Display the first few rows of the combined DataFrame to verify
combined_crime_df.head()

In [ ]:
# Create a folium map centered around Leeds
leeds_map = folium.Map(location=[53.8008, -1.5491], zoom_start=12)

# Extract the relevant columns from the crime data for latitude and longitude
crime_locations = combined_crime_df[["Latitude", "Longitude"]].dropna()

# Create a list of locations for the heatmap
heatmap_data = crime_locations.values.tolist()

# Add HeatMap to the Leeds map using green (safe) to red (dangerous) colours
HeatMap(
    heatmap_data,
    min_opacity=0.4,
    radius=10,
    blur=15,
    max_zoom=1,
    gradient={0.2: "green", 0.5: "yellow", 1: "red"},
).add_to(leeds_map)

# Overlay the Leeds boundaries from the GeoJSON file with reduced opacity
folium.GeoJson(
    leeds_gdf,
    name="Leeds Boundaries",
    style_function=lambda feature: {
        "color": "blue",
        "weight": 1,
        "opacity": 0.3,  # Reduce opacity to make streets less prominent
    },
).add_to(leeds_map)

# Save the map as an HTML file to view
leeds_map.save("output/leeds_all_crime_heatmap_reduced_street_opacity.html")

# Display the map object
leeds_map

In [ ]:
# First, reset and project GeoDataFrames appropriately
import geopandas as gpd
from shapely.geometry import Point

# Re-load the combined crime dataset and convert it into a GeoDataFrame
crime_gdf = gpd.GeoDataFrame(
    combined_crime_df,
    geometry=[
        Point(xy)
        for xy in zip(combined_crime_df["Longitude"], combined_crime_df["Latitude"])
    ],
    crs="EPSG:4326",
)

# Re-project both GeoDataFrames to a projected CRS (EPSG:27700 - British National Grid)
crime_gdf = crime_gdf.to_crs(epsg=27700)
leeds_gdf = leeds_gdf.to_crs(epsg=27700)

# Create a spatial index for the Leeds streets GeoDataFrame to optimize the matching process
leeds_sindex = leeds_gdf.sindex

# Reset crime counts in leeds_gdf
leeds_gdf["crime_count"] = 0

# Iterate through each crime and assign it to the nearest street within a reasonable threshold
max_distance_threshold = 1000  # 1 km in meters
for crime_idx, crime in crime_gdf.iterrows():
    # Get potential matches for the crime point
    possible_matches_index = list(leeds_sindex.intersection(crime.geometry.bounds))
    possible_matches = leeds_gdf.iloc[possible_matches_index]

    # Calculate distances to find the nearest street
    if not possible_matches.empty:
        distances = possible_matches.distance(crime.geometry)
        nearest_street_idx = distances.idxmin()

        # Update crime count if the distance is within the threshold
        if distances[nearest_street_idx] <= max_distance_threshold:
            leeds_gdf.at[nearest_street_idx, "crime_count"] += 1

# Re-project leeds_gdf back to the original geographic CRS for folium mapping (EPSG:4326)
leeds_gdf = leeds_gdf.to_crs(epsg=4326)

# Determine crime count thresholds for categorizing streets
safe_threshold = 5
warning_threshold = 15


# Function to determine the colour for each street based on crime count
def crime_rate_colour(crime_count):
    if pd.isna(crime_count) or crime_count <= safe_threshold:
        return "green"  # Safe
    elif crime_count <= warning_threshold:
        return "yellow"  # Warning
    else:
        return "red"  # Dangerous


# Overlay the Leeds boundaries from the GeoJSON file with crime rate-based colouring
def crime_rate_style_function(feature):
    crime_count = feature["properties"].get("crime_count", 0)
    color = crime_rate_colour(crime_count)
    return {"color": color, "weight": 3, "opacity": 0.7}


# Create a folium map with streets coloured based on crime rate
leeds_map_crime_colours = folium.Map(location=[53.8008, -1.5491], zoom_start=12)

# Add the styled GeoJSON layer to the map
folium.GeoJson(
    leeds_gdf, name="Leeds Streets Crime Rate", style_function=crime_rate_style_function
).add_to(leeds_map_crime_colours)

# Save the map as an HTML file to view
leeds_map_crime_colours.save("output/leeds_streets_crime_rate_map.html")

# Provide the link to download and view the map
leeds_map_crime_colours